In [ ]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install Qiskit
!{sys.executable} -m pip install qiskit_ibm_runtime
!{sys.executable} -m pip install matplotlib
!{sys.executable} -m pip install pylatexenc
!{sys.executable} -m pip install seaborn
!{sys.executable} -m pip install qiskit_aer

## Collision operator

This notebook is based on following publications:
```
Budinski, Ljubomir. "Quantum algorithm for the advection–diffusion equation simulated with the lattice Boltzmann method." Quantum Information Processing 20.2 (2021): 57.
Wawrzyniak, David, et al. "A quantum algorithm for the lattice-Boltzmann method advection-diffusion equation." Computer Physics Communications 306 (2025): 109373.
```



The linearized collision operator in the classical LBM for solving the 1D advection-diffusion equation can be written as a diagonal operator acting on the respective population

$
A_{i}=\begin{bmatrix}
    w_i\left(1+\frac{\mathbf{c}_iu_0}{c_s^2}\right) & 0 & 0\\ 
    0 &  \ddots & 0 \\
    0 & 0 & w_i\left(1+\frac{\mathbf{c}_iu_{M-1}}{c_s^2}\right)
\end{bmatrix}.
$

This operator is non-unitary and therefore it is not possible to construct a quantum operator straightforward.
First we combine all $A_{i}$ operators into a larger operator $U_{non\_unitary}$

$
U_{non\_unitary}=\begin{bmatrix}
    A_0 & 0 & 0\\ 
    0 &  A_1 & 0 \\
    0 & 0 & A_{2}
\end{bmatrix},
$

where we assume the density is initialized three times in a vector.
Every non-unitary operator can be put in a larger unitary operator such that

$
U_{unitary}=\begin{bmatrix}
    U_{non\_unitary} & * \\ 
    * &  *
\end{bmatrix},
$

holds.
We can see that the non-unitary evolution is computed in a subspace of a larger vector.
We can implement this operator using the Linear Combination of Unitaries algorithm.
It is defined such that

$
U_{non\_unitary}=1/2(U_1+U_2),
$

with $U_1 = U_{non\_unitary}+i\sqrt{I-U_{non\_unitary}^2}$ and $U_2= U_{non\_unitary}-i\sqrt{I-U_{non\_unitary}^2}$ being unitary matrices

Although the sum is still the non-unitary operator, it is composed of a sum of unitary operators which are implementable.

## Collision Operator

The linearized collision operator in the classical Lattice Boltzmann Method (LBM) for solving the 1D advection-diffusion equation is written as a diagonal operator acting on the respective populations:

$
A_{i}=
\begin{bmatrix}
    w_i\left(1+\frac{\mathbf{c}_iu_0}{c_s^2}\right) & 0 & 0\\ 
    0 &  \ddots & 0 \\
    0 & 0 & w_i\left(1+\frac{\mathbf{c}_iu_{M-1}}{c_s^2}\right)
\end{bmatrix}.
$

This operator is non-unitary, meaning it cannot be directly implemented as a quantum operator. To address this, all $A_{i}$ operators are combined into a larger non-unitary operator:

$
U_{\text{non\_unitary}}=
\begin{bmatrix}
    A_0 & 0 & 0\\ 
    0 &  A_1 & 0 \\
    0 & 0 & A_{2}
\end{bmatrix},
$

where we assume the density is initialized three times in a vector.

### Linear Combination of Unitaries (LCU)
Using the Linear Combination of Unitaries algorithm, $U_{\text{non\_unitary}}$ can be expressed as:

$
U_{\text{non\_unitary}} = \frac{1}{2}(U_1 + U_2),
$

where:

$
U_1 = U_{\text{non\_unitary}} + i\sqrt{I - U_{\text{non\_unitary}}^2}, \quad
U_2 = U_{\text{non\_unitary}} - i\sqrt{I - U_{\text{non\_unitary}}^2}.
$

Both $U_1$ and $U_2$ are unitary matrices, making them implementable on quantum hardware, while their sum remains non-unitary.

### Embedding into a Unitary Operator
Every non-unitary operator can be embedded into a larger unitary operator such that:

$
U_{\text{unitary}}=
\begin{bmatrix}
    U_{\text{non\_unitary}} & * \\ 
    * & *
\end{bmatrix}.
$

This embedding ensures that the non-unitary evolution occurs within a subspace of the larger unitary operator.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.quantum_info import Statevector
from qiskit import *
from qiskit.circuit.library import DiagonalGate

Initialize variables for the quantum circuit and the lattice boltzmann method

In [ ]:
#Define initial condition

Number_of_qubits = 5
Number_of_distributions = 3
Number_of_distribution_qubits = np.ceil(np.log2(Number_of_distributions))


Total_number_of_cells = 2**Number_of_qubits

mid_index = Total_number_of_cells//2

# initial density
density = np.ones(Total_number_of_cells)*0.1
density[mid_index-3 : mid_index+3] = 0.3

# we require the density three times
density_vector = np.tile(density, 3)
density_vector = np.concatenate((density_vector, np.zeros_like(density)))
normalization = np.linalg.norm(density_vector)
# normalize density vector
density_vector_normalized = density_vector / normalization

#macroscopic velocity
velocity = np.ones(Total_number_of_cells)*0.1

# weights of the distributions
weights = [4/6,1/6,1/6]
# lattice speed of sound
c_s = 1/np.sqrt(3)
# microscopic velocities
c = [0, 1, -1]

Here we compute the classical collision operator which when applied on the density vector computes the collision operator.

This is a flattened version of U_{non\_unitary}.

In [ ]:
U_non_unitary = np.zeros((Number_of_distributions, Total_number_of_cells))
for ii in range(Number_of_distributions):
    U_non_unitary[ii, :] = weights[ii]*(1+c[ii]*velocity/c_s**2)

U_non_unitary_flattened = U_non_unitary.flatten()

We compute the unitary operators $U_1$ and $U_2$, and then combine them into a larger unitary matrix. To ensure compatibility with the $2^n$ state vector, the combined unitary is padded with entries of $1$. 

$U_1 = U_{non\_unitary\_flattened}+i\sqrt{I-U_{non\_unitary\_flattened}^2}$ and $U_2= U_{non\_unitary\_flattened}-i\sqrt{I-U_{non\_unitary\_flattened}^2}$

This process increases the dimension of the original operator.

In [ ]:
#Rewrite as a sum of unitaries.  quantum_collision_operator = 1/2(Unitary_1 + Unitary_2)

#### CODE MISSING HERE
#compute U1 and U2
Unitary_1 = U_non_unitary_flattened + 1j*np.sqrt(1-U_non_unitary_flattened**2)
Unitary_2 = U_non_unitary_flattened - 1j*np.sqrt(1-U_non_unitary_flattened**2)
#####

unitary_collision_operator = np.concatenate([Unitary_1,np.ones_like(density), Unitary_2,np.ones_like(density)])

## Collision Operator Implementation

The quantum circuit is initialized with a third register containing one ancillary qubit. This ancillary qubit accounts for the increased dimension and is used exclusively for the execution of the collision operator.

### Initial State
The state vector is initially:

$
\lvert \Psi \rangle = \lvert 0 \rangle_{\text{anc}} \lvert \psi \rangle,
$

where $\lvert \psi \rangle$ combines the spaces of the \texttt{qx} and \texttt{qf} registers.

### Applying Hadamard Gate
A Hadamard gate is applied to the ancillary qubit, evolving the state vector to:

$
\lvert \Psi \rangle = \frac{1}{\sqrt{2}} (\lvert 0 \rangle_{\text{anc}} \lvert \psi \rangle + \lvert 1 \rangle_{\text{anc}} \lvert \psi \rangle).
$

### Applying Diagonal Gate
The collision operator is implemented using a diagonal gate initialized as:

$
\texttt{diagonal\_gate = DiagonalGate(unitary\_collision\_operator.tolist())}.
$

This gate is applied to the entire quantum circuit:

$
\texttt{qc.append(diagonal\_gate, qc.qubits)}.
$

This evolves the state vector to:

$
\lvert \Psi \rangle = \frac{1}{\sqrt{2}} (U_1 \lvert 0 \rangle_{\text{anc}} \lvert \psi \rangle + U_2 \lvert 1 \rangle_{\text{anc}} \lvert \psi \rangle),
$
where $U_1$ and $U_2$ are unitary matrices derived from the collision operator.

### Final Hadamard Gate
Applying another Hadamard gate to the ancillary qubit evolves the state vector to:

$
\lvert \Psi \rangle = 
\frac{1}{2} ((U_1 + U_2) \lvert 0 \rangle_{\text{anc}} \lvert \psi \rangle + (U_2 - U_1) \lvert 1 \rangle_{\text{anc}} \lvert \psi \rangle).
$

### Non-Unitary Evolution in Subspace
The term:

$
\frac{1}{2} (U_1 + U_2) \lvert 0 \rangle_{\text{anc}} \lvert \psi \rangle
$

represents the non-unitary evolution embedded in the subspace of the ancillary qubit when it is in the $\lvert 0 \rangle_{\text{anc}}$ state.

In [ ]:
#Define quantum circuit
qx = QuantumRegister(Number_of_qubits, 'qx')
qf = QuantumRegister(Number_of_distribution_qubits, 'qf')
anc = QuantumRegister(1, 'anc')
qc = QuantumCircuit(qx, qf, anc)

#### CODE MISSING HERE
# Initialize diagonal gate
diagonal_gate = DiagonalGate(unitary_collision_operator.tolist())
####

qc.initialize(density_vector_normalized, [qx,qf])

#### CODE MISSING HERE
# Apply Hadamard gates and ancilla gate to compute collision
qc.h(anc)
qc.append(diagonal_gate, qc.qubits)
qc.h(anc)
####


In [ ]:
qc.draw(output='mpl')

In [ ]:
Psi_real = np.real(Statevector(qc))*normalization
Psi = np.array(Statevector(qc))

Compute the classical collision operator.
$f_i = \rho w_i(1+c_iu/c_s^2)$

In [ ]:
relaxed_distribution = density_vector[:Total_number_of_cells*Number_of_distributions]*U_non_unitary_flattened

Check, if the quantum results match the classical.

In [ ]:
print(np.allclose(Psi_real[:Total_number_of_cells*Number_of_distributions], relaxed_distribution))

The density is computed as the sum of all distribution functions:

$
\rho = \sum_i f_i.
$

Since streaming has not been applied, the density should match the initialized values.

In [ ]:
Psi_reshaped = Psi_real[:Total_number_of_cells*Number_of_distributions].reshape((Number_of_distributions, Total_number_of_cells))
density_resummed = np.sum(Psi_reshaped, axis=0)
print(np.allclose(density_resummed, density))

Compute the probabilities of measuring a state of the state vector.
Use for this the variable Psi.

Note: P=$|\Psi|^2$

In [ ]:
#### CODE MISSING HERE
#compute the probabilities
Probabilities = np.abs(Psi)**2
####
print('Check if |<Psi|Psi>|^2 is equal to 1: ',np.sum(Probabilities))

The collision operator is applied only when the ancillary qubit is in the $\lvert 0 \rangle$ state. The $\lvert 1 \rangle$ state of the ancilla embeds a solution that does not correspond to the collision operator. Therefore, the non-unitary collision operator is implemented probabilistically, meaning it is successful only when a measurement of the ancillary qubit yields the $\lvert 0 \rangle$ state.

To compute the probabilities of measuring the ancilla in the $\lvert 0 \rangle$ state:
- Note that all possibilities where the ancilla is in the $\lvert 0 \rangle$ state are represented by the first half of the state vector.
- Similarly, all possibilities where the ancilla is in the $\lvert 1 \rangle$ state are represented by the second half of the state vector.

In [ ]:
####CODE MISSING HERE
# compute the probabilities of ancilla |0> and |1> states
Probabilty_to_compute_0_state = np.sum(Probabilities[:len(Probabilities)//2])
Probabilty_to_compute_1_state = np.sum(Probabilities[len(Probabilities)//2:])
####

print('Probability to compute the |0> state', Probabilty_to_compute_0_state)
print('Probability to compute the |1> state', Probabilty_to_compute_1_state)

Check your result using the sampling simulator.

In [ ]:
from qiskit.primitives import StatevectorSampler
# Apply measurement
qc.measure_all()
sampler = StatevectorSampler()

shots = 1000000

job = sampler.run([qc], shots=shots)
result = job.result()
data = result[0].data
counts = data.meas.get_counts()

In [ ]:
from qiskit.visualization import plot_histogram
plot_histogram(counts)

In [ ]:
probabilities = {state: count / shots for state, count in counts.items()}
sorted_states = sorted(probabilities.keys())
sorted_probabilities = [probabilities[state] for state in sorted_states]

In [ ]:
Probabilty_to_compute_0_state_sampling = np.sum(sorted_probabilities[:len(sorted_probabilities)//2])
Probabilty_to_compute_1_state_sampling = np.sum(sorted_probabilities[len(sorted_probabilities)//2:])

print('Probability to compute the |0> state', Probabilty_to_compute_0_state_sampling)
print('Probability to compute the |1> state', Probabilty_to_compute_1_state_sampling)

Conclusion:

- There is no safe way to evolve the simulation in time. Once the ancilla is measured in the $|1\rangle$, the whole run failed.
- Sampling out the result and reinitializing it is extremly costly